<a href="https://colab.research.google.com/github/nazalnihad/AI_for_good/blob/main/table_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
OPENAI_API_KEY=""

In [8]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)


def extract_chunk_from_query(user_query):
  text =    "Computational Thinking (CT) refers to the thought processes involved in expressing solutions as computational steps or algorithms that can be carried out by a computer. This course introduces Computational thinking which requires understanding the capabilities of computers, formulating problems to be addressed by a computer, and designing algorithms that a computer can execute. Computational thinking is at the heart of the computer science practices and is intrinsically connected with it."
  metadata ={'page': 1, 'source': '1-farm.pdf-5', 'filename': 'farm.pdf'}
  return text,metadata
    # Replace the following line with your actual logic.

# User's query
user_query =  "what is computational thinking"

# Extract a chunk of text from the user's query
extracted_chunk,metadata = extract_chunk_from_query(user_query)

# convo set
conversation = [
 {"role": "system", "content": f"You are a helpful assistant answering from the retrieved chunk based on the user query on a rag pipeline. Your task is to generate an answer based on the retrieved chunk of text: {extracted_chunk}. Be as helpful and precise as possible. Remember to include the filename {metadata['filename']} and page number {metadata['page']} in your answer. If the question is out of scope of the rag pipeline, respond with 'sorry, I can't answer this question'"},
 {"role": "user", "content": user_query},
]

# Make a request to the OpenAI chat API
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=conversation,
    max_tokens=800,
    temperature=0.7,
    seed=42
)

# Extract and print the assistant's reply
assistant_reply = completion.choices[0].message.content


In [9]:
print(assistant_reply)

Computational Thinking (CT) refers to the thought processes involved in expressing solutions as computational steps or algorithms that can be carried out by a computer. This course introduces Computational thinking which requires understanding the capabilities of computers, formulating problems to be addressed by a computer, and designing algorithms that a computer can execute. Computational thinking is at the heart of the computer science practices and is intrinsically connected with it. You can find more information in the file farm.pdf on page 1.


In [ ]:
data = {"filename": "Document1.pdf", "page_number": "42"}

data["filename"]

'Document1.pdf'

In [ ]:
# example responses
"""To achieve better prices, a gat (a group of farmers) can establish collective strength by working together to sell their produce. By collectively selling their crops,
 the gat can negotiate better prices and increase their overall revenue. This can be done by entering data on the Paani Foundation App, where each member of the gat provides
 information about their revenue. The gat must also provide supporting bills and documents to demonstrate the collective sale. By doing so, the gat can earn marks based on
 the percentage of total revenue obtained through collective sales. For more detailed information, please refer to Document1.pdf, page 42."""

# answer from gpt-3.5 turbo 1106
"""The gat is established to achieve better price realization for the crop through collective strength. By coming together to sell their produce,
 the members of the gat can leverage their collective power to negotiate better prices. This can ultimately lead to improved financial outcomes for
  the farmers. For further details, you can refer to Document1.pdf, page 42."""

# after setting seed
"""
One of the important purposes for which a gat is established is to achieve better price realization for the crop using its collective strength.
 This is done by the gat getting together to sell its produce, and every farmer must enter data pertaining to his/her revenue on the Paani Foundation App
  as part of the spardha. The sum total of the revenue of every member of the gat amounts to the total revenue of the gat. The gat must specify how much of
   the total revenue of the gat was obtained by collective sale as a gat along with supporting bills and documents. Marks will be given to the gat depending
   on the percentage of the total revenue of the gat obtained by collective sale as a gat. You can find more detailed information in Document1.pdf on page 42.
"""

"""
One of the important purposes for which a gat is established is to achieve better price realization for the crop using its collective strength.
 This is done by the gat getting together to sell its produce, and every farmer must enter data pertaining to his/her revenue on the Paani Foundation
  App as part of the spardha. The sum total of the revenue of every member of the gat amounts to the total revenue of the gat. The gat must specify how
   much of the total revenue of the gat was obtained by collective sale as a gat along with supporting bills and documents. Marks will be given to the
   gat depending on the percentage of the total revenue of the gat obtained by collective sale as a gat. This information can be found in Document1.pdf on page 42.
"""

"""
One of the important purposes for which a gat is established is to achieve better price realization for the crop using its collective strength.
 This is done by the gat getting together to sell its produce, thereby leveraging its collective power to obtain better prices. Every farmer must
  enter data pertaining to his/her revenue on the Paani Foundation App as part of the spardha. The sum total of the revenue of every member of the
   gat amounts to the total revenue of the gat. The gat must specify how much of the total revenue was obtained by collective sale as a gat, along
   with supporting bills and documents. Marks will be given to the gat depending on the percentage of the total revenue obtained by collective sale as
    a gat. This information can be found in Document1.pdf on page 42.
"""
# 800 token

"""
One of the important purposes for which a gat is established is to achieve better price realization for the crop using its collective strength.
This is done by the gat getting together to sell its produce, thereby leveraging its collective power to obtain better prices. Every farmer must
 enter data pertaining to his/her revenue on the Paani Foundation App as part of the spardha. The sum total of the revenue of every member of the
 gat amounts to the total revenue of the gat. The gat must specify how much of the total revenue was obtained by collective sale as a gat, along with
 supporting bills and documents. Marks will be given to the gat depending on the percentage of the total revenue obtained by collective sale as a gat.
  This information can be found in Document1.pdf on page 42
"""
"""
One of the important purposes for which a gat is established is to achieve better price realization for the crop using its collective strength.
This is done by the gat getting together to sell its produce. Every farmer must enter data pertaining to his/her revenue on the Paani Foundation
 App as part of the spardha. The sum total of the revenue of every member of the gat amounts to the total revenue of the gat. The gat must specify
  how much of the total revenue of the gat was obtained by collective sale as a gat along with supporting bills and documents. Marks will be given to
  the gat depending on the percentage of the total revenue of the gat obtained by collective sale as a gat.
 For more detailed information, you can refer to Document1.pdf, page 42.
"""

"""
One of the important purposes for which a gat is established is to achieve better price realization for the crop using its collective strength.
 This is done by the gat getting together to sell its produce, thereby leveraging its collective power to obtain better prices. Every farmer
  must enter data pertaining to his/her revenue on the Paani Foundation App as part of the spardha. The sum total of the revenue of every member
   of the gat amounts to the total revenue of the gat. The gat must specify how much of the total revenue was obtained by collective sale as a gat,
    along with supporting bills and documents. Marks will be given to the gat depending on the percentage of the total revenue obtained by collective sale
     as a gat. For further details, you can refer to Document1.pdf, page 42.
"""

"""
I found the relevant information in the file farm.pdf on page 1. The competition rules state that the gat
must be of farmers cultivating a single crop. Farmers in a gat may cultivate more than one crop, however all
 farmers in the gat must cultivate the crops in the same ratio.
"""

"""
I found the information you need in the file "farm.pdf" on page 1. The competition rules state that the gat must be of farmers cultivating a single
crop. Farmers in a gat may cultivate more than one crop, however all farmers in the gat must cultivate the crops in the same ratio.
"""


In [10]:
def get_assistant_reply(user_query):
   # Extract a chunk of text from the user's query
   extracted_chunk, metadata = extract_chunk_from_query(user_query)

   # convo set
   conversation = [
       {"role": "system", "content": f"You are a helpful assistant answering from the retrieved chunk based on the user query on a rag pipeline. Your task is to generate an answer based on the retrieved chunk of text: {extracted_chunk}. Be as helpful and precise as possible. Remember to include the filename {metadata['filename']} and page number {metadata['page']} in your answer. If the question is out of scope of the rag pipeline, respond with 'sorry, I can't answer this question'"},
       {"role": "user", "content": user_query},
   ]

   # Make a request to the OpenAI chat API
   completion = client.chat.completions.create(
       model="gpt-3.5-turbo-1106",
       messages=conversation,
       max_tokens=800,
       temperature=0.7,
       seed=42
   )

   # Extract and return the assistant's reply
   assistant_reply = completion.choices[0].message.content
   return assistant_reply


In [11]:
user_query = "what is computational thinking"
assistant_reply = get_assistant_reply(user_query)
print(assistant_reply)


Computational Thinking (CT) refers to the thought processes involved in expressing solutions as computational steps or algorithms that can be carried out by a computer. This course introduces Computational thinking which requires understanding the capabilities of computers, formulating problems to be addressed by a computer, and designing algorithms that a computer can execute. Computational thinking is at the heart of the computer science practices and is intrinsically connected with it. For further details, you can refer to the file farm.pdf, page 1.
